<a href="https://colab.research.google.com/github/SUNITHA123-THUMMA/Assignment-for-Full-Stack-Development-Internship/blob/main/Sentiment_analysis_API_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Train a Sentiment Analysis Model

In [1]:
# sentiment_train.py
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import pickle

# Sample dataset
data = {
    "text": [
        "I love this product!",
        "This is terrible.",
        "Amazing experience.",
        "I hate this item.",
        "It’s just okay.",
        "Very good and useful.",
        "Worst purchase ever."
    ],
    "label": [1, 0, 1, 0, 0, 1, 0]  # 1 = Positive, 0 = Negative
}
df = pd.DataFrame(data)

# Tokenization
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["text"])
sequences = tokenizer.texts_to_sequences(df["text"])
padded = pad_sequences(sequences, padding="post", maxlen=10)

# Model
model = Sequential([
    Embedding(5000, 16, input_length=10),
    LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(padded, np.array(df["label"]), epochs=10, verbose=1)

# Save model and tokenizer
model.save("model/sentiment_model.h5")
with open("model/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.4286 - loss: 0.6916
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5714 - loss: 0.6934
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5714 - loss: 0.6912
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5714 - loss: 0.6887
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.5714 - loss: 0.6862
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5714 - loss: 0.6905
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5714 - loss: 0.6831
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5714 - loss: 0.6938
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4286 - loss: 0.6942
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5714 - loss: 0.6855


Step 2: Build FastAPI Backend

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load model & tokenizer
model = tf.keras.models.load_model("model/sentiment_model.h5")
with open("model/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

app = FastAPI(title="Sentiment Analysis API")

class TextInput(BaseModel):
    text: str

@app.get("/")
def home():
    return {"message": "Welcome to the Sentiment Analysis API!"}

@app.post("/predict")
def predict_sentiment(input: TextInput):
    sequence = tokenizer.texts_to_sequences([input.text])
    padded = pad_sequences(sequence, maxlen=10, padding="post")
    prediction = model.predict(padded)[0][0]
    sentiment = "Positive" if prediction > 0.5 else "Negative"
    return {"text": input.text, "sentiment": sentiment, "score": float(prediction)}


Step 3: Requirements File

Step 4: Run API Locally

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load model and tokenizer
model = tf.keras.models.load_model("model/sentiment_model.h5")
with open("model/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Test a sample
text = "I really love this product!"
seq = tokenizer.texts_to_sequences([text])
padded = pad_sequences(seq, maxlen=10, padding="post")
prediction = model.predict(padded)[0][0]
sentiment = "Positive" if prediction > 0.5 else "Negative"

print(f"Text: {text}")
print(f"Predicted Sentiment: {sentiment} (Score: {prediction:.3f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
Text: I really love this product!
Predicted Sentiment: Negative (Score: 0.483)


FastAPI app run inside Google Colab and access it through a public URL using ngrok.

STEP 1 – Install all dependencies

In [7]:
!pip install fastapi uvicorn pyngrok tensorflow pydantic


STEP 2 – Import libraries and configure ngrok

In [8]:
from pyngrok import ngrok

# Paste your authtoken here
ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN")


STEP 3 – Create a simple FastAPI app

In [9]:
%%writefile app.py
from fastapi import FastAPI
from pydantic import BaseModel
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load model and tokenizer
model = tf.keras.models.load_model("model/sentiment_model.h5")
with open("model/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

app = FastAPI(title="Sentiment Analysis API")

class TextInput(BaseModel):
    text: str

@app.get("/")
def home():
    return {"message": "Welcome to the Sentiment Analysis API!"}

@app.post("/predict")
def predict_sentiment(input: TextInput):
    seq = tokenizer.texts_to_sequences([input.text])
    padded = pad_sequences(seq, maxlen=10, padding="post")
    pred = model.predict(padded)[0][0]
    sentiment = "Positive" if pred > 0.5 else "Negative"
    return {"text": input.text, "sentiment": sentiment, "score": float(pred)}


Writing app.py


STEP 4 – Run FastAPI + expose through ngrok

In [15]:
from pyngrok import ngrok

# Replace the below string with your real token (keep the quotes)
ngrok.set_auth_token("34e4jUd680W6sZhLTrhk3iiP0VK_62o89mY95cgpoDbGC5No3")


Re-run the FastAPI + ngrok cell

In [16]:
from pyngrok import ngrok
import subprocess

# Kill any existing tunnels (clean start)
ngrok.kill()

# Create tunnel for FastAPI app
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run the app in the background
process = subprocess.Popen(["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"])


Public URL: NgrokTunnel: "https://karry-looped-tamely.ngrok-free.dev" -> "http://localhost:8000"
